In [ ]:
import sys
sys.path.append('../')
from configuration_tiny_gpt import TinyGPTConfig
from modeling_tiny_gpt import TinyGPTForCausalLM
from src.models.dense.config import ModelConfig
import torch
from dataclasses import asdict
from transformers import AutoTokenizer
import shutil

In [ ]:
def convert_checkpoint_to_hf(
    checkpoint_path,
    output_dir,
    config_dict,
    tokenizer_name="gpt2"
):
    """Convert your checkpoint to HuggingFace format"""
    shutil.copy("modeling_tiny_gpt.py", f"{output_dir}/modeling_tiny_gpt.py")
    config = TinyGPTConfig(**config_dict)
    
    hf_model = TinyGPTForCausalLM(config)
    
    # load your checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    hf_model.model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    
    # load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    # save to HuggingFace format
    hf_model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return hf_model, tokenizer

config_dict = asdict(ModelConfig())

hf_model, tokenizer = convert_checkpoint_to_hf(
    checkpoint_path="dense_step_9000.pt",
    output_dir="./hf-tiny-gpt",
    config_dict=config_dict
)
print(hf_model)

/var/folders/16/7k9wy23d4tgf_rq3jzhdr_yc0000gn/T/ipykernel_98521/2313183862.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map

TinyGPTForCausalLM(
  (model): tiny_gpt(
    (tok_embedding): Embedding(50257, 768)
    (layers): ModuleList(
      (0-4): 5 x layer(
        (attention): SimpleMultiHeadAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=False)
          (c_proj): Linear(in_features=768, out_features=768, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ffn): SwiGLUFFN(
          (w_1): Linear(in_features=768, out_features=3072, bias=True)
          (w_2): Linear(in_features=768, out_features=3072, bias=True)
          (out): Linear(in_features=3072, out_features=768, bias=True)
        )
        (attn_norm): RMSNorm()
        (ffn_norm): RMSNorm()
      )
    )
    (norm): RMSNorm()
    (output): Linear(in_features=768, out_features=50257, bias=True)
  )
)


In [8]:
from huggingface_hub import HfApi, create_repo

def push_to_hub(
    local_dir="./hf-tiny-gpt",
    repo_name="Marmik/tiny-gpt",
    private=False
):
    
    # create repository
    create_repo(
        repo_id=repo_name,
        private=False,
        exist_ok=True
    )
    
    # initialize API
    api = HfApi()
    
    # upload all files
    api.upload_folder(
        folder_path=local_dir,
        repo_id=repo_name,
        repo_type="model"
    )
    
    print(f"Model uploaded to: https://huggingface.co/{repo_name}")

# push to hub
push_to_hub(
    local_dir="./hf-tiny-gpt",
    repo_name="Marmik/tiny-gpt",
    private=False
)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Model uploaded to: https://huggingface.co/Marmik/tiny-gpt
